# Introduction to Statistical Modeling: Linear Regression

**Overview**
- Concept of statistical modeling
- Defining statistical models with Patsy
- Linear regression
- Discrete regression: Logistic regression and Poisson model

We will use the [statsmodels](https://www.statsmodels.org/stable/index.html) libray which provides classes and functions for defining statistical models and fitting them to observed data, for calculating descriptive statistics and carrying out statistical tests. The api modules collect the publically accessible symbols that the library provides.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg

The [Patsy](https://patsy.readthedocs.io/en/latest/) library allows us to write statistical models as simple formulas. It is inspired by statiscal software such as R and S.   The statmodels library internally uses the Patsy library and thus we don't need to access the Patsy's functions directly. But we will use Patsy for demonstration.

In [ ]:
import patsy

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats

**Main Problem:** For a set of response(dependent) variables $y$, and explanatory(independent) variable $x$, we want to find a relationship (model) between $y$ and $x$:
- mathematical model:         $~~~ y = f(x)$
- statistical model:        $~~~ y = f(x) + \epsilon~~$ where $\epsilon$ is a random variable. A model is statistical when the data ${y_i, x_i}$ has an element of uncertainty (e.g. due to measurement noise) which is described as $\epsilon$.


A widely used model is 
$$
y = \beta_0 + \beta_1 x + \epsilon,
$$

where $\beta_0$ and $\beta_1$ are model parameters and $\epsilon$ is normally distributed with $0$ mean and variance $\sigma^2$. 
* If $x$ is a scalar, the model is known as *simple linear regression*.
* If $x$ is a vector, the model is known as *multiple linear regression*.
* If $y$ is a vector, the model is known as *multivariate linear regression*.


After we form the model, we construct the so-called design matrices $y$ and $X$ such that the regression problem can be written in matrix form:

$$
  y = X\beta + \epsilon,
$$

where $y$ is the vector(or matrix) of observations, $\beta$ is a vector of coefficients and $\epsilon$ is the residual(error).

**Example:**

Suppose the observed values are
$$y = [1,2,3,4,5]$$
and there are two independent variables with values
$$x_1 = [6,7,8,9,10]$$
and
$$ x_2 = [11,12,13,14,15]. $$

Assume we use the linear model
$$ y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2 $$
(Note: *linear* with respect to the $\beta$ coefficients.)

Therefore, the design matrix is
$$ X = [1, x_1, x_2, x_1 x_2]. $$

Here is Python/NumPy to implement this

In [ ]:
y = np.array([1,2,3,4,5])
x1 = np.array([6,7,8,9,10])
x2 = np.array([11,12,13,14,15])
X = np.vstack([np.ones(5),x1,x2,x1*x2]).T
X

Given $X$ and $y$, we can solve for $\beta$ using least-squares method:

In [ ]:
beta, res, rank, svals = np.linalg.lstsq(X,y,rcond=-1)

In [ ]:
print(beta)

In the above example, constructing the design matrix $X$ was fairly simple. However, it can be more difficult for more complicated models. 

The `Patsy` library provides a simple [formula language](https://patsy.readthedocs.io/en/latest/formulas.html#the-formula-language)  to handle this.

First, we create a dictionary that maps the variable names in the model to the corresponding data arrays:

\[This is very similar to how in Sympy we associated symbol names with Python variables.\]

In [ ]:
data = {"y":y, "x1":x1, "x2":x2}
print(data)
print(data["x2"])

To use the model  
$$~ y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2$$ 
with Patsy, we can use the formula `y ~ 1 + x1 + x2 + x1*x2` 
* note we leave out the coefficients

From the formula we can easily get the design matrices:

In [ ]:
import patsy
# y, X = patsy.dmatrices("y ~ 1 + x1 + x2 + x1*x2")
y, X = patsy.dmatrices("y ~ x1*x2")

In [ ]:
# Look at y and compare with input
y

In [ ]:
# Look at X and compare with manual construction
X

We can also use the ordinary linear regression (OLS) class in the `statsmodels` library (instead of `np.linalg.lstsq`) to solve for the parameter vector:

In [ ]:
model = sm.OLS(y, X)
result = model.fit()

In [ ]:
result

In [ ]:
#compare this with the answer from np.linalg.lstsq
result.params

We can skip the step of creating the design matrices by using the statmodels formula API (we imported it as `smf`) by the following:

In [ ]:
model = smf.ols("y ~ x1*x2", data)
result = model.fit()
result.params

This saves us time when we want to add and remove terms in the model. 

**Exercise:** Instead of using a Python dictionary, put `y`, `x1` and `x2` into a Pandas data frame, and solve using the `statsmodels` library.

In [ ]:
y = np.array([1,2,3,4,5])
data = {"y":y, "x1":x1, "x2":x2}
data

In [ ]:
#look at C01 or the Pandas doc for how to make a data frame from a dictionary with data
df = pd.DataFrame(data)
model = smf.ols("y ~ x1*x2", df)
result = model.fit()
result.params

### (Simplified) summary of the Patsy formula syntax

|Syntax|Example| Description |
|:-|:- |:---|
|lhs ~ rhs|y ~ x <br>(equivalent to y ~ 1+x) |~ is used to separate LHS (dependent variables) and <br> RHS (independent variables) |
|var * var| x1 * x2 <br>(equivalent to 1+x1+x2+x1*x2) |An interaction term that implicitly contains all lower-order terms|
|var + var| x1 + x2 <br>(equivalent to y ~ 1+x1+x2) |+ denotes the union of terms |
|var - var| x1 - x2 <br> |- removes the following term |
|var:var| x1:x2 |: denotes a pure interaction term (e.g. $x_1\cdot x_2$)|

For a complete syntax, see the Patsy [documentation](https://patsy.readthedocs.io/en/latest/).

In [ ]:
# Repeat above to show we can just specify x1*x2 (look at the 
# design matrices as well as at the solution)

In [ ]:
model = smf.ols("y ~ x1*x2", data)
result = model.fit()
result.params

# Linear Regression
**Basic workflow for analyzing a statistic model using statsmodels**:
1. Create an instance of model class, for example, using `mod = sm.MODEL(y,X)` or `mod = smf.model(formula, data)` where `MODEL` and `model` are the names of a particular model (e.g. OLS, GLS, Logit, etc)
2. Fit the model to the data:  `result = model.fit()`
3. Print summary statistics for the result:  `result.summary()`
4. Post-process the model fit results by methods and attributes `params`, `resid`, `fittedvalues`, `predict`
5. Visualize the result by Matplotlib or `statsmodels.graphics` module.


### Example (linear regression):
Consider fitting a model to generated data whose true value is $ y = 1 + 2x_1 + 3x_2 + 4x_1 x_2$.

* Sample 100 random data points in \[-2,2\] for our independent variables. 

In [ ]:
N = 1000  # We will repeat with N=1000 and 10000
np.random.seed(12345) # so we all get the same values
x1 = 4*(np.random.random(N)-0.5)
x2 = 4*(np.random.random(N)-0.5)

* Define a function to compute y_true,
* insert corresponding column of values into the data frame, then
* examine the frame

In [ ]:
def y_true(x1,x2):
    return 1 + 2*x1 + 3*x2 + 4*x1*x2

y = y_true(x1,x2)
df = pd.DataFrame({"y":y, "x1":x1, "x2":x2})
df

Add normal-distributed noise to the true values and store the result in the "y" column of the data frame


In [ ]:
noise = 0.1*np.random.randn(N)
df["y"] += noise


In [ ]:
df.head()

**1st model:** $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2$ 

**Question:** what is the simplest corresponding Patsy model?  Replace the `????` below with the model.

In [ ]:
# Step1: Create an instance of model class (fit the model to the data using ordinary least square)
model = smf.ols("y ~ x1 + x2", df)

In [ ]:
# Step2: Fit the model to the data
result = model.fit()

In [ ]:
# Step3: Print summary statistics for the result
result.summary()

*What to look for:*
- R-squared:  indicates how well the model fits the data. The value is between 0 and 1. The value 1 corresponds to a perfect fit.  
- The `coef` column contains the model parameters. 
- *t-statistics*:   $t$ = coef/(std err).  The greater $|t|$, the more likely that the corresponding coefficient is non-zero (which means that it has a significant predictive power).
   - [Recall: the greater $|t|$, the greater the evidence against the null hypothesis. Here the null hypothesis is that the coefficient is $0$.]
- p-value:  small p-value (<0.05 ??) indicates that that coefficient is more likely to be non-zero.
    - [Recall: small p-value means strong evidence against the null hypothesis.]
- 95% range 

Summary:  
* R-squared close to 1 => good fit.
* High $t$ or small $p$-value => that coeff is significantly different from $0$.
* 95% range of parameter gives sense of how well-defined is the value (or how sensitive is the fit to changes in the value)

In [ ]:
# We can also get the R-squared directly:
result.rsquared

Note that by using ordinary least-square regression we assume that the residuals (of the fitted model and the data) is normally distributed.  Before analyzing data, we might not know if this condition is met. However, we can investigate this by using statistical tests (with null hypothesis that the residuals are normally distributed) and/or plotting the residual.

In [ ]:
# we can look at the residual 
result.resid

In [ ]:
plt.plot(result.resid)
plt.show()

In [ ]:
# Check the normality of the residual
z, p = stats.normaltest(result.resid.values)
print(p)
z, p = stats.shapiro(result.resid.values)
print(p)

Here (with 100 points) the $p$-value is not especially large from the `normaltest` nor especially small from the `shapiro` test, so it is unclear if we should reject the null hypothesis (i.e. the assumption that the residuals are normally-distributed is correct). 
* Recall that we're considering our first model ($y = \beta_0 + \beta_1 x_1 + \beta_2 x_2$) which does not include the $x_1 . x_2$ term, so we were likely (or at least I was) expecting the error distribution to be skewed and not normally distributed.  
* This just underscores that these tests are just guides and cannot be used in isolation.
* With more points the tests become more definitive

Let's look at the distribution of the residual

In [ ]:
plt.hist(result.resid,bins=20); # change to 50 bins with more points
plt.show()

We can also use a graphical method (`qqplot`) to check for normality. The [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot) compares the observed distribution of values between quantiles with that expected theoretically from the normal distribution. If the distribution is normal, you'll get the straight line $y = x$ (i.e., a line through the origin at 45 degrees assuming both axes are on the same scale).

Note: I don't really like these plots since how straight is straight enough or how close to 45 degrees?


In [ ]:
?smg.qqplot

In [ ]:
# Use the QQ graphical method to check for normality.
fig, ax = plt.subplots(figsize=(8, 4))
smg.qqplot(result.resid, ax=ax, fit=True, line="45") # need fit to get scales of x and y to match
fig.tight_layout()
plt.show()

This is some evidence that the first model is not sufficient.  

Let's add the interaction term:

**2nd model:** $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2 $   **.  

In [ ]:
# Repeat the steps from the previous analysis (Steps 1-3) ... again, enter the correct Patsy formula
model = smf.ols("y ~ x1*x2", df)
result = model.fit()
result.summary()

The r-squared is very close to 1, indicating a nearly perfect fit.

Then we look at the residuals and check if the residuals are normally distributed:

In [ ]:
plt.plot(result.resid) # You'll see the residuals are much smaller
plt.show()

In [ ]:
#1. statisical test
z, p = stats.normaltest(result.resid.values)
print(p) # we want this to be large
z, p = stats.shapiro(result.resid.values)
p

Both p values are now significant --- so we accept the null hypothesis and conclude that the residuals are probably normally distributed

Let's look ...

In [ ]:
plt.hist(result.resid,bins=20)
plt.show()

In [ ]:
#2. qq-plot 
fig, ax = plt.subplots(figsize=(8, 4))
smg.qqplot(result.resid, ax=ax, fit=True, line="45")

fig.tight_layout()
plt.show()

Repeat with 1000 and 10,000 points to see how the two QQ plots differ with more data.

If we are happy with the fitted model, we can extract the model parameters:

In [ ]:
result.params

Given values of the indepedent variables ($x_1$, and $x_2$ in this case), we can use the `predict` method to get the prediction (the $y$ value).

Compute the predictions on a 50x50 mesh between -2 and 2.

In [ ]:
x = np.linspace(-2,2,50)
X1, X2 = np.meshgrid(x,x)
X1 = X1.ravel()
X2 = X2.ravel()
print(X1)
print(X2)


In [ ]:
newdata = pd.DataFrame({"x1":X1, "x2":X2})
ynew = result.predict(newdata)
ynew.describe()

In [ ]:
type(ynew)

In [ ]:
ynew = np.array(ynew)

In [ ]:
# the result is a vector
ynew.shape

In [ ]:
# so we must reshape it into a 50x50 grid/mesh/matrix
ynew = ynew.reshape(50,50)
ynew.shape

In [ ]:
# also reshape the X1 and X2 vectors to a square matrix
X1 = X1.reshape(50,50)
X2 = X2.reshape(50,50)

In [ ]:
# plot the true data and the fitted model
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

def plot_y_contour(ax, Y, title):
    c = ax.contourf(X1, X2, Y, 15, cmap=plt.cm.RdBu)
    ax.set_xlabel(r"$x_1$", fontsize=20)
    ax.set_ylabel(r"$x_2$", fontsize=20)
    ax.set_title(title)
    cb = fig.colorbar(c, ax=ax)
    cb.set_label(r"$y$", fontsize=20)

plot_y_contour(axes[0], y_true(X1, X2), "true relation")
plot_y_contour(axes[1], ynew, "fitted model")

fig.tight_layout()
plt.show()

### Datasets from R
The statmodels provides an interface to load data sets to explore.  See http://www.statsmodels.org/dev/datasets/index.html#available-datasets  for available data sets.

As an example, we will load a dataset named "Icecream" from the package "Ecdat":

In [ ]:
dataset = sm.datasets.get_rdataset("Icecream", "Ecdat")

In [ ]:
dataset.data

In [ ]:
dataset.data.temp

We see that this dataset has 4 variables: cons(consumption), income, price, and temp. 

**Exercise:** Model the consumption as a linear model with price and temperature as independent variables without an intercept/constant term (i.e., forcing the intercept to be zero):

In [ ]:
model = smf.ols("cons ~ price + temp - 1", dataset.data)  # again enter the Patsy formula `cons ~ price + temp - 1`
result = model.fit()
result.summary()

In [ ]:
# Graphical tools like plot_fit (regression plot) can give a quick look at our fitted model
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

smg.plot_fit(result, 0, ax=ax1)
smg.plot_fit(result, 1, ax=ax2)

fig.tight_layout()
plt.show()

The consumption seems linearly correlated to the temp but doesn't seem so on the price (it's perhaps because the price range is quite small). 

## References: 
- *Numerical Python: A Practical Techniques Approach for Industry*  by Robert Johansson (Chapter 14)